# Descripción del Dataset

## Variables no utilizadas

### Id (primera columna sin nombre)

Identificación del registro en el dataset.

- Tipo de variable: numérica.
- No se utilizará como variable porque no aporta información sobre el diamante en sí, y facilitaría el sobreentrenamiento de los modelos que utilicemos.

---

## Variables de entrada

### Quilate (carat)

Peso en quilates del diamante. Un quilate equivale a 0,2 gramos. Junto al corte, color y claridad, forman lo que se denomina "4C": el conjunto de características más importantes a la hora de evaluar la calidad de un diamante.

- Tipo de variable: numérica.
- Transformaciones: Para ciertos modelos será necesario escalar los datos.

### Corte (cut)

Etiqueta que indica la calidad del corte del diamante. El corte es la más importante de las "4C" porque es la característica que tiene mayor influencia en el brillo del diamante.

![Diamond Cut](images/diamond_cut.gif)

- Tipo de variable: categórica.
- Etiquetas posibles (orden creciente): Fair, Good, Very Good, Premium, Ideal.
- Transformaciones: Para ciertos modelos será necesario aplicar One Hot Encoding.

### Color (color)

Color del diamante. Es la segunda característica más importante de un diamante, debido a que es lo que el ojo nota luego del destello. El grado de color del diamante, en realidad, se refiere a la ausencia de color en el mismo.

![Diamond Color](images/diamond_color.png)

- Tipo de variable: categórica.
- Etiquetas posibles (orden creciente): J, I, H, G, F, E, D.
- Transformaciones: Para ciertos modelos será necesario aplicar One Hot Encoding.

### Claridad (clarity)

Etiqueta que indica qué tan visibles son las imperfecciones y/o inclusiones del diamante. Las imperfecciones son pequeñas características de la superficie del diamante, mientras que las inclusiones se refieren a características internas del mismo. 
La claridad es la menos importante de las 4C. Esto se debe a que las inclusiones son microscópicas y, por ende, no afectan la belleza del diamante de una forma visible.

![Diamond Clarity](images/diamond_clarity.jpg)

- Tipo de variable: categórica.
- Etiquetas posibles (orden creciente): I3, I2, I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF, FL. No hay ejemplos en el dataset de claridad tipo I3, I2 y FL.
- Transformaciones: Para ciertos modelos será necesario aplicar One Hot Encoding.

### Porcentaje de profundidad (depth)

Porcentaje de profundidad obtenido dividiendo la altura de un diamante, medida desde el culet hasta la tabla, por su diámetro de filetín promedio. Contribuye a la apariencia, el brillo y el fuego del diamante. También permite que el diamante parezca más grande.

![Diamond Parts](images/diamond_parts.gif)

- Tipo de variable: numérica.
- Transformaciones: Para ciertos modelos será necesario escalar los datos.

### Tabla (table)

El ancho de la tabla expresado como un porcentaje del diámetro promedio del diamante. Medida esencial para el rendimiento ante la luz de un diamante.

- Tipo de variable: numérica.
- Transformaciones: Para ciertos modelos será necesario escalar los datos.

### Longitud (x)

Longitud en milímetros del diamante.

- Tipo de variable: numérica.
- Transformaciones: Para ciertos modelos será necesario escalar los datos.

### Ancho (y)

Ancho en milímetros del diamante.

- Tipo de variable: numérica.
- Transformaciones: Para ciertos modelos será necesario escalar los datos.

### Profundidad (z)

Profundidad en milímetros del diamante.

- Tipo de variable: numérica.
- Transformaciones: Para ciertos modelos será necesario escalar los datos.

---

## Variable de salida

### Precio (price)

Precio del diamante expresado en dólares (EEUU).

- Tipo de variable: categórica.
- Transformaciones: 
    1. Crear una nueva columna, llamada "price_category", que representa las etiquetas correspondientes al rango de precio al que pertenezcan los precios de los diamantes. **Esta nueva columna será la variable de salida a predecir**.
    2. Para ciertos modelos será necesario aplicar One Hot Encoding.
- Etiquetas posibles: Menor a 1000, Entre 1000 y 3000, Entre 3000 y 6000, Entre 6000 y 10000, Mayor a 10000.

---

# Descripción de los datos

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Parámetros para los gráficos.
plt.rcParams['figure.figsize'] = 16, 6
TITLE_FONTSIZE = 18
LABEL_FONTSIZE = 12

In [ ]:
# Cargar dataset.
dataset = pd.read_csv('data/diamonds.csv', index_col=0)

#### Forma del dataset

In [ ]:
filas, columnas = dataset.shape
print(f'El dataset tiene {filas} filas y {columnas} columnas.')

#### Primeras filas del dataset

In [ ]:
dataset.sort_values(['price']).head()

In [ ]:
dataset.sort_values(['price']).tail()

#### Métricas de las variables numéricas del dataset

In [ ]:
dataset.describe()

#### Cantidad de valores nulos por atributo del dataset

In [ ]:
dataset.isnull().sum().to_frame('Cantidad de valores Nulos')

#### Histograma de precios

In [ ]:
ax = dataset.price.plot.hist(bins=50, title='Histograma de precios', grid=True)
ax.title.set_size(TITLE_FONTSIZE)
ax.set_xlabel('Precio (Dólares)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Frecuencia', fontsize=LABEL_FONTSIZE)
ax.set_axisbelow(True)

#### Transformación de la variable de salida Precio

In [ ]:
dataset.price.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).to_frame('Precio')

In [ ]:
price_category_labels = [
    'Menor a 1000', 
    'Entre 1000 y 3000', 
    'Entre 3000 y 6000', 
    'Entre 6000 y 10000', 
    'Mayor a 10000'
]
price_limits = [0, 1000, 3000, 6000, 10000, 20000]

dataset['price_category'] = pd.cut(dataset.price, price_limits, labels=price_category_labels)

In [ ]:
dataset.head()

---

# Análisis detallado de variables

### Balanceo de la variable de salida

In [ ]:
ax = dataset.price_category.value_counts().plot.pie(autopct='%1.0f%%', figsize=(10,10))
ax.set_ylabel('')
ax.set_title('Balanceo de etiquetas de Precios', fontsize=TITLE_FONTSIZE);

Como se observa en el gráfico, en general, la proporción de instancias en cada categoría es similar y no debería afectar a los modelos clasificadores que utilicemos. 

La excepción se encuentra marcada en las etiquetas "Entre 6000 y 10000" y "Mayor a 10000". Se decidió efectuar esta subdivisión ya que, si bien la proporción era adecuada para la división (22%), el rango de precios que comprendía era demasiado amplio y perdía sentido interpretativo.

---

## Comportamiento de variables de entrada y relación con variable de salida

### Quilates

#### Comportamiento de la variable

In [ ]:
dataset.carat.describe().to_frame('carat')

In [ ]:
ax, data = dataset.boxplot(column=['carat'], return_type='both', vert=False, figsize=(16, 4))
ax.set_title('Distribución de los valores de la variable Quilates', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Quilate', fontsize=LABEL_FONTSIZE)
ax.set_yticklabels([]);

In [ ]:
outliers = len(data['fliers'][0].get_xdata())
print(f'Cantidad de valores atípicos: {outliers}')
print(f'Porcentaje de valores atípicos: {outliers / dataset.shape[0] * 100:.2f}%')

Se observa que el peso de los diamantes en el dataset varía entre 0,2 y 5,01 quilates. La mayor concentración de datos se halla por debajo de 1,04 quilates.

#### Relación con variable de salida

In [ ]:
ax = dataset.boxplot(column=['carat'], by='price_category')
ax.set_title('Distribución de quilates según categoría de precio', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Categoría de Precio', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Quilates', fontsize=LABEL_FONTSIZE)
ax.get_figure().suptitle('');

In [ ]:
carat_dataframe = pd.DataFrame()
carat_dataframe['price'] = dataset['price']
carat_limits = [0, 0.4, 0.6, 0.8, 1, 2, 3, 4, 5, 6]
carat_dataframe['carat_range'] = pd.cut(dataset.carat, carat_limits)
ax = carat_dataframe.groupby(['carat_range',]).mean()[['price',]].plot.bar(rot=0, legend=False, grid=True)
ax.set_title('Precio promedio por rango de quilates', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Rango de Quilates', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Precio promedio', fontsize=LABEL_FONTSIZE)
ax.set_axisbelow(True)

En general, se evidencia que el precio de los diamantes aumenta conforme lo hace su peso.


### Corte

#### Comportamiento de la variable

In [ ]:
ax = dataset.cut.value_counts().plot.pie(autopct='%1.0f%%', figsize=(10,10));
ax.set_title('Balanceo de tipos de Corte', fontsize=TITLE_FONTSIZE)
ax.set_ylabel('');

Dado que el corte es una de las características más importantes de un diamante, es poco usual encontrar cortes de baja calidad (como 'Fair' y 'Good').

#### Relación con variable de salida

In [ ]:
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
ax = dataset.groupby(['cut', 'price_category',]).size().unstack().reindex(cut_order).T.plot.bar(stacked=True,
                                                                                                rot=0, grid=True)
ax.set_title('Proporción de tipo de corte por categoría de precio', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Categoría de Precio', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Cantidad', fontsize=LABEL_FONTSIZE)
ax.legend(title='Tipo de corte')
ax.set_axisbelow(True)

In [ ]:
ax = dataset.groupby(['cut',]).mean()[['price']].reindex(cut_order).plot.bar(rot=0, grid=True, legend=False)
ax.set_title('Precio promedio por tipo de corte', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Tipo de Corte', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Precio', fontsize=LABEL_FONTSIZE)
ax.set_axisbelow(True)

Si bien el corte forma parte de las "4C", los gráficos exhibidos demuestran que, por sí solo, no es un factor tan determinante del precio, como si lo es el peso de la piedra. 

### Color

#### Comportamiento de la variable

In [ ]:
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D', ]
ax = dataset.color.value_counts().reindex(color_order).plot.pie(autopct='%1.0f%%', figsize=(10,10));
ax.set_title('Balanceo de tipos de Color', fontsize=TITLE_FONTSIZE)
ax.set_ylabel('');

El color es una de las cuatro características principales de un diamante. La ausencia de color es fundamental para un mejor aspecto, y se recomienda comprar piedras casi incoloras (H o superior) o incoloras (F o superior), lo que explica su mayor proporción en el dataset utilizado.

#### Relación con variable de salida

In [ ]:
ax = dataset.groupby(['color', 'price_category',]).size().unstack().reindex(color_order).T.plot.bar(stacked=True, 
                                                                                                    rot=0, 
                                                                                                    grid=True)
ax.set_title('Proporción de tipo de color por categoría de precio', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Categoría de Precio', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Cantidad', fontsize=LABEL_FONTSIZE)
ax.legend(title='Tipo de Color')
ax.set_axisbelow(True)

In [ ]:
ax = dataset.groupby(['color',]).mean()[['price']].reindex(color_order).plot.bar(rot=0, grid=True, legend=False)
ax.set_title('Precio promedio por tipo de color', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Tipo de Color', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Precio', fontsize=LABEL_FONTSIZE)
ax.set_axisbelow(True)

En el primer gráfico se observa que la proporción de tipo de color es similar en todas las categorías de precios.

Por otro lado, en el segundo gráfico se evidencia que el precio promedio de un diamante con un tipo de color de menor calidad es superior a una piedra con mejor tipo de color. Si bien esto podría parecer contrario a la lógica, podemos suponer que, dado que los tipos de color D, E y F son mejores y, por ende, más costosos, los mismos se comercializan en forma de piedras más pequeñas y de menor peso. En el gráfico presentado a continuación podemos validar la hipótesis formulada.

In [ ]:
ax = dataset.groupby(['color',]).mean()[['carat']].reindex(color_order).plot.bar(rot=0, grid=True, legend=False)
ax.set_title('Peso promedio por tipo de color', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Tipo de Color', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Quilates', fontsize=LABEL_FONTSIZE)
ax.set_axisbelow(True)

### Claridad

#### Comportamiento de la variable

In [ ]:
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
ax = dataset.clarity.value_counts().reindex(clarity_order).plot.pie(autopct='%1.0f%%', figsize=(10,10))
ax.set_ylabel('')
ax.set_title('Balanceo de tipo de Claridad', fontsize=TITLE_FONTSIZE);

El dataset presenta una mayor proporción de diamantes con claridad media-alta. Esto se debe a diversos factores:

- Resulta inusual encontrar piedras de calidad FL, que no presenten imperfecciones ni inclusiones de ningún tipo.
- No es común el comercio de piedras de claridad baja.
- A pesar de ser una de las características principales para un diamante, la claridad no resulta tan importante como las demás. Esto se debe a que, por lo general, las inclusiones no son visibles sin un factor de magnificación de 10x.

#### Relación con variable de salida

In [ ]:
ax = dataset.groupby(['clarity', 'price_category',]).size().unstack().reindex(clarity_order).T.plot.bar(
    stacked=True, rot=0, grid=True);
ax.set_title('Proporción de tipo de claridad por categoría de precio', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Categoría de Precio', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Cantidad', fontsize=LABEL_FONTSIZE)
ax.legend(title='Tipo de Claridad')
ax.set_axisbelow(True)

In [ ]:
ax = dataset.groupby(['clarity',]).mean()[['price']].reindex(clarity_order).plot.bar(
    rot=0, grid=True, legend=False)
ax.set_title('Precio promedio por tipo de claridad', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Tipo de Claridad', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Precio', fontsize=LABEL_FONTSIZE)
ax.set_axisbelow(True)

In [ ]:
ax = dataset.groupby(['clarity',]).mean()[['carat']].reindex(clarity_order).plot.bar(
    rot=0, grid=True, legend=False)
ax.set_title('Peso promedio por tipo de claridad', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Tipo de Claridad', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Quilates', fontsize=LABEL_FONTSIZE)
ax.set_axisbelow(True)

Como puede verse, la relación de la variable Claridad con la variable de salida es análoga a la relación entre Color y la Categoría de Precio. 

### Porcentaje de profundidad

#### Comportamiento de la variable

In [ ]:
dataset.depth.describe().to_frame('depth')

In [ ]:
ax, data_depth = dataset.boxplot(column=['depth'], return_type='both', vert=False, figsize=(16, 4))
ax.set_title('Distribución de los valores de la variable Porcentaje de Profundidad', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('% de Profundidad', fontsize=LABEL_FONTSIZE)
ax.set_yticklabels([]);

In [ ]:
outliers_depth = len(data_depth['fliers'][0].get_xdata())
print(f'Cantidad de valores atípicos: {outliers_depth}')
print(f'Porcentaje de valores atípicos: {outliers_depth / dataset.shape[0] * 100:.2f}%')

Puede observarse que el porcentaje de profundidad de los diamantes del dataset varía entre un mínimo de 43% y un máximo de 79%. La mayor concentración de datos se halla entre 61% y 62,5%.

Los expertos en diamantes consideran que cualquier valor entre 56,5% y 65% es bueno. Sin embargo, el porcentaje de profundidad ideal se halla entre el 59,5% y el 62,9%. Esto explica la distribución del dataset.

#### Relación con variable de salida

In [ ]:
ax = dataset.boxplot(column=['depth'], by='price_category')
ax.set_title('Distribución de % de Profundidad según categoría de precio', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Categoría de Precio', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('% de Profundidad', fontsize=LABEL_FONTSIZE)
ax.get_figure().suptitle('');

In [ ]:
depth_dataframe = pd.DataFrame()
depth_dataframe['price'] = dataset['price']
depth_limits = [0, 50, 60, 61, 61.5, 62, 62.5, 62.9, 80]
depth_dataframe['depth_range'] = pd.cut(dataset.depth, depth_limits)
ax = depth_dataframe.groupby(['depth_range',]).mean()[['price',]].plot.bar(rot=0, legend=False, grid=True)
ax.set_title('Precio promedio por rango de % de profundidad', fontsize=TITLE_FONTSIZE)
ax.set_xlabel('Rango de % de Profundidad', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Precio promedio', fontsize=LABEL_FONTSIZE)
ax.set_axisbelow(True)

El porcentaje de profundidad, si bien resulta una característica importante, no afecta tanto al valor del precio como las "4C".
Puede concluirse que esta medida se encuentra bastante "estandarizada", con una gran proporción de diamantes en un pequeño rango de porcentaje de profundidad.